In [ ]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import requests
import json
import folium
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
# 따릉이 서버에 post 방식으로 요청을 해서 서버가 응답하는 데이터를 받는다.
targetSite = 'https://www.bikeseoul.com/app/station/getStationRealtimeStatus.do'
request = requests.post(targetSite, data={
    'stationGrpSeq': 'ALL'
})
print(request)
print(type(request.text)) # <class 'str'>
print(request.text)

In [ ]:
# 따릉이 서버가 응답한 json 형태의 문자열을 파이썬에서 사용하기 위해서 딕셔너리 타입으로 변환한다.
# json 모듈의 loads() 메소드 사용
bike_json = json.loads(request.text) # 'bike_json = request.json()'와 같다. => requests 모듈의 json 메소드 사용
print(type(bike_json)) # json을 사용했더니 <class 'dict'>로 바뀜
print(bike_json.keys())
print(bike_json)

In [ ]:
bike_json['realtimeList'] 
# bike_json.get('realtimeList') 
# bike_json.('realtimeList') X => get을 꼭 써줘야 하고, 안 쓰는 건 pd에서 가능

In [ ]:
# pandas.io.json 모듈의 json_normalize() 메소드로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# 데이터가 많을 때는 pd가 편하기 때문!
# json_normalize(딕셔너리, 데이터프레임으로 변환할 데이터가 할당된 딕셔너리의 key)
bike_df = json_normalize(bike_json, 'realtimeList')
bike_df

In [ ]:
bike_df.columns

In [ ]:
# stationId': 대여소 id
# stationName: 대여소 이름
# stationLongitude: 대여소 경도
# stationLatitude: 위도
# rackTotCnt: 주차 가능한 자전거 대여수
# parkingBikeTotCnt: 주차된 따릉이 LCD형 대수 => 사용하지 않음
# parkingQRBikeCnt: 주차된 따릉이 QR형 대수 => 일반 따릉이
# parkingELECBikeCnt: 주차된 새싹 따릉이 대수

bike_df_map = bike_df[['stationId', 'stationName', 'stationLongitude', 'stationLatitude', 'parkingQRBikeCnt', 'parkingELECBikeCnt']]
bike_df_map

In [ ]:
# bike_df_map.dtypes
# => 타입 확인 결과 object(=Str)
bike_df_map.info() 

In [ ]:
bike_df_map['stationLongitude'] = bike_df_map['stationLongitude'].astype(float)
bike_df_map['stationLatitude'] = bike_df_map['stationLatitude'].astype(float)
bike_df_map['parkingQRBikeCnt'] = bike_df_map['parkingQRBikeCnt'].astype(int)
bike_df_map['parkingELECBikeCnt'] = bike_df_map['parkingELECBikeCnt'].astype(int)
bike_df_map['parkingTotalBikeCnt'] = bike_df_map['parkingQRBikeCnt'] + bike_df_map['parkingELECBikeCnt']
bike_df_map

In [ ]:
bike_df_map.info() 

In [ ]:
# bike_df_map.to_csv('./data/bike_df_map.csv', index=False) # csv파일 내릴 때는 꼭 index=False 넣기(인덱스가 데이터로 착각할 수 있어서)

Geo Coding  

지오코딩 => 주소로 위도와 경도 얻기  
역지오코딩 => 위도와 경도로 주소 얻기  
!pip install geopy  

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
# 주소를 인수로 넘겨받아 위도와 경도를 리턴하는 함수
def geocoding(address):
    geolocoder = Nominatim(user_agent='South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    return {'위도': geo.latitude, '경도': geo.longitude}

# address = geocoding('서울시 관악구 봉천동')
# address = geocoding('서울시 종로구 삼일대로') # 더 자세한 주소 정보는 불가
# print(address)

In [ ]:
# 위도와 경도를 넘겨받아 주소를 리턴하는 함수
def geocoding_reverse(lat_lot):
    geolocoder = Nominatim(user_agent='South Korea', timeout=None)
    return geolocoder.reverse(lat_lot)
'''
address = geocoding_reverse('37.484665,126.9484961')
# print(type(address))
# print(address)
addr = str(address).split(', ')
# print(type(addr))
# ['서울신봉초등학교', '양녕로6길', '봉천동', '중앙동', '관악구', '서울', '08729', '대한민국']
print(addr[-3], addr[-4], addr[-6])

address = geocoding_reverse('37.54018026,126.97359281')
addr = str(address).split(', ')
# ['한강대로', '남영동', '용산구', '서울', '04353', '대한민국']
print(addr[-3], addr[-4], addr[-5])

address = geocoding_reverse('37.1497727,127.0770233')
addr = str(address).split(', ')
# ['동부대로', '오산시', '경기도', '18132', '대한민국']
print(addr[-3], addr[-4], addr[-5])
'''
pass

In [ ]:
bike_df_map['goo'] = np.NaN
bike_df_map['dong'] = np.NaN
bike_df_map

In [ ]:
# goo와 dong에 위에서 얻은 위도와 경도를 이용해서 주소 넣기
# 2718 rows × 9 columns의 형태이므로 shape을 통해 행과 열을 돌릴 수 있다.
'''
for i in range(bike_df_map.shape[0])[:]:
    lat_lot = '{},{}'.format(bike_df_map.loc[i, 'stationLatitude'], bike_df_map.loc[i, 'stationLongitude'])
    # print(lat_lot)
    address = geocoding_reverse(lat_lot) # 위 함수로 올라가서 위도와 경도 얻어온다.
    addr = str(address).split(', ') # 얻어온 위도와 경도를 , 단위로 분리
    # print(addr[-4], addr[-5]) # 그 후 뒤에서 4번째, 5번째 값 출력
    try:
        bike_df_map.loc[i, 'goo'] = addr[-4]
    except:
        print('{}번째 인덱스 스테이션의 -4번째 주소가 없음'.format(i))
    try:
        bike_df_map.loc[i, 'dong'] = addr[-5]
    except:
        print('{}번째 인덱스 스테이션의 -5번째 주소가 없음'.format(i))

bike_df_map
'''
pass

In [ ]:
# bike_df.map.to.csv('./data/bike_20230907.csv', index=False)
bike_df_map = pd.read_csv('./data/bike_20230907.csv', encoding='cp949')
bike_df_map

In [ ]:
bike_df_map[bike_df_map.dong.isnull() | bike_df_map.dong.isnull()]

In [ ]:
bike_df_map[bike_df_map.goo.isnull() | bike_df_map.dong.isnull()].stationId

In [ ]:
errorList = list(bike_df_map[bike_df_map.goo.isnull() | bike_df_map.dong.isnull()].stationId)
print(type(errorList)) # list
print(errorList) 

In [ ]:
for error in errorList:
    bike_error = bike_df_map[bike_df_map.stationId == error]
    lat_lot = '{},{}'.format(bike_error.iloc[0, 3], bike_error.iloc[0, 2])
    # print(bike_error, lat_lot)
    
    address = geocoding_reverse(lat_lot)
    addr = str(address).split(', ')
    # print(addr)
    bike_df_map.loc[bike_error.index, 'goo'] = addr[-3]
    bike_df_map.loc[bike_error.index, 'dong'] = addr[-4]
bike_df_map[bike_df_map[bike_df_map.goo.isnull() | bike_df_map.dong.isnull()]

In [ ]:
# bike_df.map.to.csv('./data/bike_20230907.csv', index=False)

In [ ]:
bike_map = folium.Map(location=[bike_df_map.stationLatitude.mean(), bike_df_map.stationLongitude.mean()], zoom_start=13)
for index, data in bike_df_map.iterrows():
    station = data.stationName.split('.')
    if len(station) == 1:
        stationName = station[0].strip()
    else:
        stationName = station[1].strip()
    # if절을 삼항연산자 사용해서 한 줄로 줄일 수 있다.
    # stationName = station[0].strip() if len(station) == 1 else station[1].strip()

    string = '{}. 일반: {}대, 새싹: {}대'.format(stationName, data.parkingQRBikeCnt, data.parkingELECBikeCnt)
    # print(string)
    stationInfo = folium.Popup(string, max_width=300)
    folium.Marker(location=[data.stationLatitude, data.stationLongitude], popup=stationInfo,
                 icon=folium.Icon(color='green', icon='hand-down')).add_to(bike_map)
bike_map.save('./output/bike.html')
bike_map

종로구 따릉이 스테이션 위치

In [ ]:
bike_df_map_goo = bike_df_map[bike_df_map.goo == '종로구']
bike_df_map_goo

In [ ]:
bike_map = folium.Map(location=[bike_df_map_goo.stationLatitude.mean(), bike_df_map_goo.stationLongitude.mean()], zoom_start=14)
for index, data in bike_df_map_goo.iterrows():
    station = data.stationName.split('.')
    stationName = station[0].strip() if len(station) == 1 else station[1].strip()

    string = '{}. 일반: {}대, 새싹: {}대'.format(stationName, data.parkingQRBikeCnt, data.parkingELECBikeCnt)
    stationInfo = folium.Popup(string, max_width=300)
    folium.Marker(location=[data.stationLatitude, data.stationLongitude], popup=stationInfo,
                 icon=folium.Icon(color='green', icon='hand-down')).add_to(bike_map)
bike_map.save('./output/bike.map.html')
bike_map